In [19]:
import torch
batch = torch.load('/root/atomsurf/debug_spike_step_6299/batches/batch_step_06299.pt')
print(f"Batch size: {batch['num_graphs']}")
print(f"Labels: {batch['label']}")

Batch size: 4
Labels: tensor([2, 1, 1, 5])


In [20]:
batch.keys()

dict_keys(['label', 'lig_coord', 'num_graphs', 'pocket_names', 'surface', 'graph'])

In [22]:
import os
import pickle

# Load training dataset
systems = pickle.load(open('/root/atomsurf/data/masif_ligand/raw_data_MasifLigand/splits/train.p', 'rb'))
pocket_names = list(systems.keys())

print(f"Total training samples needed: {len(pocket_names)}")

# Check surface files
surface_dir = '/root/atomsurf/data/masif_ligand/surf_1.0_False'
surface_files = os.listdir(surface_dir)
print(f"Surface files available: {len(surface_files)}")

# Check how many pockets have surfaces
missing = 0
found = 0

for pocket in pocket_names[:10000]:  # Check first 100
    surf_file = os.path.join(surface_dir, f"{pocket}.pt")
    if os.path.exists(surf_file):
        found += 1
    else:
        missing += 1

print(f"\nFirst 100 samples:")
print(f"  Found: {found}")
print(f"  Missing: {missing}")
print(f"  Missing rate: {missing/(found+missing)*100}%")

# Show sample names
print(f"\nSample pocket name: {pocket_names[0]}")
print(f"Expected file: {os.path.join(surface_dir, pocket_names[0] + '.pt')}")
print(f"Exists: {os.path.exists(os.path.join(surface_dir, pocket_names[0] + '.pt'))}")

ModuleNotFoundError: No module named 'numpy._core'

In [23]:
import os
import pickle

# What's in the training dataset
systems = pickle.load(open('/root/atomsurf/data/masif_ligand/raw_data_MasifLigand/splits/train.p', 'rb'))
pocket_names = list(systems.keys())

# What surfaces exist
surface_dir = '/root/atomsurf/data/masif_ligand/surf_1.0_False'
surface_files = set([f.replace('.pt', '') for f in os.listdir(surface_dir)])

# Find the mismatch
missing = []
for pocket in pocket_names:
    if pocket not in surface_files:
        missing.append(pocket)

print(f"Training pockets: {len(pocket_names)}")
print(f"Surface files available: {len(surface_files)}")
print(f"Missing surfaces: {len(missing)}")
print(f"\nMissing rate: {len(missing)/len(pocket_names)*100:.1f}%")

ModuleNotFoundError: No module named 'numpy._core'

In [16]:
import os
import pickle

# Load training pockets
systems = pickle.load(open('/root/atomsurf/data/masif_ligand/raw_data_MasifLigand/splits/train.p', 'rb'))
pocket_names = list(systems.keys())

# Get surface files
surface_dir = '/root/atomsurf/data/masif_ligand/surf_1.0_False'
surface_files = [f.replace('.pt', '') for f in os.listdir(surface_dir)]

# Sample comparison
print("Sample training pocket names:")
for i in range(5):
    print(f"  {pocket_names[i]}")

print("\nSample surface file names:")
for i in range(5):
    print(f"  {surface_files[i]}")

# Check one specific missing pocket
missing_example = None
for pocket in pocket_names:
    if pocket not in surface_files:
        missing_example = pocket
        break

if missing_example:
    print(f"\nExample missing pocket: {missing_example}")
    
    # Check for similar names
    similar = [s for s in surface_files if missing_example.split('_')[0] in s]
    print(f"Similar surface files found: {len(similar)}")
    if similar:
        print(f"Examples:")
        for s in similar[:3]:
            print(f"  {s}")

Sample training pocket names:
  3O7W_A_patch_0_SAM
  4YTP_ACBD_patch_0_FAD
  4YTP_ACBD_patch_1_HEM
  4YMP_A_patch_0_HEM
  4IVM_B_patch_0_FAD

Sample surface file names:
  3OZV_A_patch_0_HEM
  6FQY_AB_patch_0_NAP
  4XQ9_AB_patch_1_NAD
  4DCA_A_patch_0_ADP
  2C2C_A_patch_0_HEM

Example missing pocket: 3O7W_A_patch_0_SAM
Similar surface files found: 0


In [8]:
import os
import pickle

# What preprocessing processes (strip .npz)
patch_dir = '/root/atomsurf/data/masif_ligand/dataset_MasifLigand'
all_patches_raw = os.listdir(patch_dir)
all_patches = set([p.replace('.npz', '') for p in all_patches_raw])

# What training uses
systems = pickle.load(open('/root/atomsurf/data/masif_ligand/raw_data_MasifLigand/splits/train.p', 'rb'))
train_pockets = set(systems.keys())

# Check overlap
overlap = all_patches.intersection(train_pockets)
in_preprocessing_not_training = all_patches - train_pockets
in_training_not_preprocessing = train_pockets - all_patches

print(f"All patches (from dataset_MasifLigand): {len(all_patches)}")
print(f"Training pockets (from train.p): {len(train_pockets)}")
print(f"\nOverlap: {len(overlap)}")
print(f"In preprocessing but not in training: {len(in_preprocessing_not_training)}")
print(f"In training but not in preprocessing: {len(in_training_not_preprocessing)}")

if in_training_not_preprocessing:
    print(f"\nFirst 5 missing from preprocessing:")
    for p in list(in_training_not_preprocessing)[:5]:
        print(f"  {p}")

All patches (from dataset_MasifLigand): 2254
Training pockets (from train.p): 1972

Overlap: 1634
In preprocessing but not in training: 620
In training but not in preprocessing: 338

First 5 missing from preprocessing:
  1PFK_ACBD_patch_6_ADP
  1S4E_A_patch_0_ADP
  2OYY_ABIKJL_patch_2_HEM
  2PO0_ACBEDF_patch_1_ADP
  5KF9_AB_patch_1_COA
